In [3]:
import logging
# ロギングの設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Seleniumのセットアップ
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(service=service, options=options)

# 指定されたURLからHTMLを取得
url = 'https://store.starbucks.co.jp/pref/tokyo/'
driver.get(url)

# ページ全体をスクロールしてJavaScriptによる動的コンテンツをロード
SCROLL_PAUSE_TIME = 2  # 待機時間を2秒に設定

while True:
    try:
        # 「もっと見る」ボタンを探してクリック
        more_button = driver.find_element(By.CLASS_NAME, 'searching-result__show-more__button')
        more_button.click()
        time.sleep(SCROLL_PAUSE_TIME)  # ページが更新されるのを待つ
    except Exception as e:
        # 「もっと見る」ボタンが見つからない場合、ループを終了
        break

# 最後にページ全体をスクロールして全コンテンツをロード
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# ページの内容を取得
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

# 店舗情報を格納するためのリスト
starbucks_list = []

# 店舗情報が含まれる要素をすべて取得
store_items = soup.find_all('li', class_='searching-result__item')

for item in store_items:
    # 店舗名を取得
    title_element = item.find('p', class_='result__item__info__title')
    title = title_element.text.strip() if title_element else None
    
    # 住所を取得
    address_element = item.find('p', class_='result__item__info__desc')
    address = address_element.text.strip() if address_element else None

    # データリストに情報を追加
    if title and address:
        starbucks_list.append({
            'store_name': title,
            'address': address,
        })

# データフレームに変換して表示
df = pd.DataFrame(starbucks_list)
print(df.head(50))

2024-05-21 14:55:31,302 - INFO - ====== WebDriver manager ======
2024-05-21 14:55:31,775 - INFO - Get LATEST chromedriver version for google-chrome
2024-05-21 14:55:31,844 - INFO - Get LATEST chromedriver version for google-chrome
2024-05-21 14:55:31,900 - INFO - Driver [/Users/emiyoneda/.wdm/drivers/chromedriver/mac64/124.0.6367.207/chromedriver-mac-x64/chromedriver] found in cache


                       store_name  \
0                   皇居外苑 和田倉噴水公園店   
1                       竹橋パレスサイド店   
2                        大手町プレイス店   
3                          大手町ビル店   
4                    大手町東京サンケイビル店   
5                      ＫＤＤＩ大手町ビル店   
6                       グランスタ丸の内店   
7                       グランスタ八重洲店   
8                      ＪＲ東京駅日本橋口店   
9                         丸の内オアゾ店   
10               ＪＲ東海 東京駅新幹線南ラチ内店   
11                       グランスタ東京店   
12            東京駅八重洲北口 東京ギフトパレット店   
13                      丸の内新東京ビル店   
14                       丸の内三菱ビル店   
15            東京ステーションシティ サピアタワー店   
16                   東京ミッドタウン日比谷店   
17                      有楽町メトロピア店   
18                     JR有楽町駅京橋口店   
19                        日比谷シティ店   
20                  赤坂プルデンシャルタワー店   
21                      山王パークタワー店   
22                        霞ダイニング店   
23  TSUTAYA BOOKSTORE MARUNOUCHI店   
24                         丸の内ビル店   
25                          新丸ビル店   
2

In [25]:
#DBファイルとして保存する関数
def save_to_database(quotes, db_name=None):
    if db_name is None:
        date_str = datetime.now().strftime("%Y%m%d_%H%M%S")
        db_name = f'quotes_{date_str}.db'
    
    try:
        df = pd.DataFrame(quotes)
        conn = sqlite3.connect(db_name)
        df.to_sql('quotes', conn, if_exists='append', index=False)
        conn.close()
        logging.info(f"Data successfully saved to {db_name}")
    except Exception as e:
        logging.error(f"Error saving data to database: {e}")

In [26]:
# データベース名を指定
db_name = 'starbucks_list.db'  # ここに実際のデータベースファイル名を入力してください

save_to_database(df, db_name)

2024-05-21 14:58:09,679 - INFO - Data successfully saved to starbucks_list.db
